In [1]:
import geotransformer

In [2]:
import multi_part_assembly

In [3]:
import shape_assembly

In [4]:
import jhutil
import yaml
import torch

In [5]:
torch.multiprocessing.set_start_method('spawn')

In [6]:
from multi_part_assembly.datasets.geometry_data import build_geometry_dataset, build_geometry_dataloader
from multi_part_assembly.datasets.geometry_data import GeometryPartDataset as BreakingBadDataset
import jhutil
import torch
from torch.utils.data import Dataset, DataLoader
import torch

In [7]:
cfg = jhutil.load_yaml("../yamls/data_example.yaml")

In [8]:
train_data, val_data = build_geometry_dataset(cfg)

In [9]:
tmp = train_data.get_original_pcs(216)

0000  "artifact/653931_sf/mode_16"


In [10]:

import jhutil;jhutil.jhprint(1111, tmp)

1111  "[tensor[36559, 3] n=109677 (0.4Mb) x∈[-0.500, 0.500] μ=-0.050 σ=0.204, tensor[9272, 3] n=27816 (0.1Mb) x∈[-0.147, 0.165] μ=0.008 σ=0.075, tensor[1707, 3] n=5121 (20Kb) x∈[-0.060, 0.251] μ=0.094 σ=0.080, tensor[1955, 3] n=5865 (23Kb) x∈[-0.098, 0.251] μ=0.042 σ=0.115, tensor[505, 3] n=1515 (5.9Kb) x∈[-0.097, 0.251] μ=0.061 σ=0.106, tensor[0, 3] n=0 \u001bempty\u001b[0m]"
